In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns

from functools import partial
from multiprocessing import Pool

from PIL import Image

In [2]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.python.platform import gfile
import os.path
import re
import sys
import tarfile
from subprocess import Popen, PIPE, STDOUT
def run(cmd):
  p = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)
  return p.stdout.read()

In [3]:
import urllib.request
def cbk(a,b,c):
  per = 100.0*a*b/c
  if(per >100):
    per = 100
  print(per)
urllib.request.urlretrieve("http://45.32.248.250:8000/train.zip", "/tmp/train.zip",cbk)
urllib.request.urlretrieve("http://45.32.248.250:8000/test.zip", "/tmp/test.zip",cbk)

In [4]:
dbutils.fs.mv("file:/tmp/train.zip", "dbfs:/tmp/sample/train.zip")
dbutils.fs.mv("file:/tmp/test.zip", "dbfs:/tmp/sample/test.zip")
display(dbutils.fs.ls("dbfs:/tmp/sample"))

In [5]:
display(dbutils.fs.ls("dbfs:/tmp/sample/train.zip_files/train/"))

In [6]:
import zipfile  
def un_zip(file_name):  
    """unzip zip file"""  
    zip_file = zipfile.ZipFile(file_name)  
    if os.path.isdir(file_name + "_files"):  
        pass  
    else:  
        os.mkdir(file_name + "_files")  
    for names in zip_file.namelist():  
        zip_file.extract(names,file_name + "_files/")  
    zip_file.close()  
un_zip('/dbfs/tmp/sample/train.zip')

In [7]:
un_zip('/dbfs/tmp/sample/test.zip')

In [8]:
train_df = pd.read_csv("/dbfs/FileStore/tables/train.csv")
dbutils.fs.mkdirs("/tmp/sample/json/")
train_df.head()

In [9]:
def mkSubFile(lines,head,srcName,sub):  
    [des_filename, extname] = os.path.splitext(srcName)  
    filename  = des_filename + '_' + str(sub) + extname  
    print( 'make file: %s' %filename)  
    fout = open(filename,'w')  
    try:  
        fout.writelines([head])  
        fout.writelines(lines)  
        return sub + 1  
    finally:  
        fout.close()  
  
def splitByLineCount(filename,count):  
    fin = open(filename,'r')  
    try:  
        head = fin.readline()  
        buf = []  
        sub = 1  
        for line in fin:  
            buf.append(line)  
            if len(buf) == count:  
                sub = mkSubFile(buf,head,filename,sub)  
                buf = []  
        if len(buf) != 0:  
            sub = mkSubFile(buf,head,filename,sub)     
    finally:  
        fin.close()  
  
if __name__ == '__main__':  
    splitByLineCount('/dbfs/FileStore/tables/train.csv',1000)  

#Stuructured Streaming

In [11]:
from pyspark.sql.types import *

inputPath = "dbfs:/FileStore/tables/"

# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
csvSchema = StructType([ StructField("name", StringType(), True), StructField("type", StringType(), True) ])

# Static DataFrame representing data in the JSON files
staticInputDF = (
  spark
    .read
    .schema(csvSchema)
    .option("header","true")
    .csv(inputPath)
)

display(staticInputDF)

In [12]:
staticCountsDF = (
  staticInputDF
    .groupBy(
       staticInputDF.type
    )    
    .count()
)
staticCountsDF.cache()

# Register the DataFrame as table 'static_counts'
staticCountsDF.createOrReplaceTempView("static_counts")

In [13]:
%sql select type, sum(count) as total_count from static_counts group by type

In [14]:
from pyspark.sql.functions import *

# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(csvSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .csv(inputPath)
)

# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.type
    )
    .count()
)

# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

In [15]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only in Spark 2.0)
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)

In [16]:
from time import sleep
sleep(5)

In [17]:
%sql select type as time, count from counts order by type

In [18]:
sleep(5)

In [19]:
%sql select type as time, count from counts order by type

In [20]:
sleep(5)

In [21]:
%sql select type as time, count from counts order by type

In [22]:
%sql select type, sum(count) as total_count from counts group by type order by type

For this fixed two-column dataset, I will put csv as a static dataframe. We also have another way to input the csv, and register datagframe as “count”. Features and distribution of dataset could be easily attached. We found that the dataset distributed unevenly, and most of categories have limited photo as the training set. It would be difficult for us to set a model and implement deep learning.

#Deep Learning with Apache Spark and TensorFlow

In [25]:
import numpy as np
import pandas as pd
import seaborn as sns
from glob import glob
from PIL import Image
import matplotlib.pylab as plt

train_images = glob('/dbfs/tmp/sample/train.zip_files/train/*jpg')
df = pd.read_csv('/dbfs/FileStore/tables/train.csv')

df["Image"] = df["Image"].map( lambda x : "/tmp/sample/train.zip_files/train/"+x)
ImageToLabelDict = dict( zip( df["Image"], df["Id"]))
df.to_csv('/dbfs/tmp/sample/file.csv')
df["Image"] = df["Image"].map( lambda x : "/dbfs"+x)
df.to_csv('/dbfs/tmp/sample/filewithdbfs.csv')

In [26]:
from keras.applications import InceptionV3
import h5py
model = InceptionV3(weights="imagenet")
model.compile(optimizer='rmsprop', loss='mse')
model.save('/tmp/model-full.h5')

In [27]:
from sparkdl import readImages, DeepImagePredictor
from sparkdl.image import imageIO as imageIO

train_images_df = imageIO.readImages("/tmp/sample/train.zip_files/train/")
train_df = train_df.withColumnRenamed('Image','filepath')
train_df = train_df.join(train_images_df,'filepath')
train_df.show()

In [28]:
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
import os
from sparkdl import KerasImageFileTransformer
from pyspark.sql.types import StringType
def loadAndPreprocessKerasInceptionV3(uri):
    # this is a typical way to load and prep images in keras
    image = img_to_array(load_img(uri, target_size=(64, 64)))
    image = np.expand_dims(image, axis=0)
    return preprocess_input(image)

transformer = KerasImageFileTransformer(inputCol="Image", outputCol="predictions",
                                        modelFile="/tmp/model-full.h5",
                                        imageLoader=loadAndPreprocessKerasInceptionV3,
                                        outputMode="vector")

dirpath = "/dbfs/tmp/sample/train.zip_files/train/"
files = [os.path.abspath(os.path.join(dirpath, f)) for f in os.listdir("/dbfs/tmp/sample/train.zip_files/train/") if f.endswith('.jpg')]
uri_df = sqlContext.createDataFrame(files, StringType()).toDF("Image")
csv_df = spark.read.csv("/tmp/sample/filewithdbfs.csv",header=True)

In [29]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

csv_df = spark.read.csv("/tmp/sample/filewithdbfs.csv",header=True)
stringIndexer = StringIndexer(inputCol="Id", outputCol="categoryIndex")
indexed_dateset = stringIndexer.fit(csv_df).transform(csv_df)
encoder = OneHotEncoder(inputCol="categoryIndex", outputCol="categoryVec")
image_dataset = encoder.transform(indexed_dateset)
image_dataset.show(5,False)

In [30]:
from sparkdl.estimators.keras_image_file_estimator import KerasImageFileEstimator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

def loadAndPreprocessKerasInceptionV3(uri):
    # this is a typical way to load and prep images in keras
    image = img_to_array(load_img(uri, target_size=(64, 64)))
    image = np.expand_dims(image, axis=0)
    return preprocess_input(image)


"""def load_image_and_process(uri):
  import PIL.Image
  from keras.applications.imagenet_utils import preprocess_input
  original_image = PIL.Image.open(uri).convert('RGB')
  resized_image = original_image.resize((64, 64), PIL.Image.ANTIALIAS)
  image_array = np.array(resized_image).astype(np.float32)
  image_tensor = preprocess_input(image_array[np.newaxis, :])
  return image_tensor"""

myEstimator = KerasImageFileEstimator(inputCol='Image',outputCol='CategoryIndex', modelFile='/tmp/model-full.h5',imageLoader=loadAndPreprocessKerasInceptionV3, kerasFitParams={"epochs": 5, "batch_size": 64})


In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

kerasParams1 = {'batch_size':10, 'epochs':10}
kerasParams2 = {'batch_size':5, 'epochs':20}

myParamMaps = \
  ParamGridBuilder() \
    .addGrid(myEstimator.kerasFitParams, [kerasParams1, kerasParams2]) \
    .build()

cv = CrossValidator(estimator = myEstimator, evaluator = MulticlassClassificationEvaluator, estimatorParamMaps = myParamMaps)
cvModel = cv.fit(image_dataset)
kerasTransformer = cvModel.bestModel  # of type KerasTransformer

We had tried many different ways to fit the dataset with the existing model in Apache Spark, unfortunately, most of them could not complete the process perfectly. They always face some problems at some specific points, including miss some specific type of data, cannot extract any features from our dataset or incompatible with multi-thread. Pickle seems not work so after refer to professor, we will leave the exsiting running result here and turn to traditional way to implement our neutral network. The below descripes our progress. Because of the performance provided by community edition of databricks and the file number limitation, I will actually run locally instead of in databricks.

In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
from glob import glob
from PIL import Image
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import os
from subprocess import check_output
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

In [34]:
train_images = glob("/dbfs/tmp/sample/train.zip_files/train/*jpg")
df = pd.read_csv("/dbfs/FileStore/tables/train.csv")

df["Image"] = df["Image"].map( lambda x : "/dbfs/tmp/sample/train.zip_files/train/"+x)
ImageToLabelDict = dict( zip( df["Image"], df["Id"]))

In [35]:
SIZE = 64
#image are imported with a resizing and a black and white conversion
def ImportImage( filename):
    img = Image.open(filename).convert("LA").resize( (SIZE,SIZE))
    return np.array(img)[:,:,0]
train_img = np.array([ImportImage( img) for img in train_images])
x = train_img

In [36]:
print( "%d training images" %x.shape[0])

print( "Nbr of samples/class\tNbr of classes")
for index, val in df["Id"].value_counts().value_counts().sort_index().iteritems():
    print( "%d\t\t\t%d" %(index,val))

In [37]:
class LabelOneHotEncoder():
    def __init__(self):
        self.ohe = OneHotEncoder()
        self.le = LabelEncoder()
    def fit_transform(self, x):
        features = self.le.fit_transform( x)
        return self.ohe.fit_transform( features.reshape(-1,1))
    def transform( self, x):
        return self.ohe.transform( self.la.transform( x.reshape(-1,1)))
    def inverse_tranform( self, x):
        return self.le.inverse_transform( self.ohe.inverse_tranform( x))
    def inverse_labels( self, x):
        return self.le.inverse_transform( x)

y = list(map(ImageToLabelDict.get, train_images))
lohe = LabelOneHotEncoder()
y_cat = lohe.fit_transform(y)

In [38]:
#use of an image generator for preprocessing and data augmentation
x = x.reshape( (-1,SIZE,SIZE,1))
input_shape = x[0].shape
x_train = x.astype("float32")
y_train = y_cat

image_gen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True)

#training the image preprocessing
image_gen.fit(x_train, augment=True)

#visualization of some images out of the preprocessing
augmented_images, _ = next( image_gen.flow( x_train, y_train.toarray(), batch_size=4*4))

Hyperparameter-1

In [40]:
batch_size = 128
num_classes = len(y_cat.toarray()[0])
epochs = 40

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

model = Sequential()
model.add(Conv2D(48, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(48, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(48, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.33))
model.add(Flatten())
model.add(Dense(24, activation='relu'))
model.add(Dropout(0.33))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()
model.fit_generator(image_gen.flow(x_train, y_train.toarray(), batch_size=batch_size),
          steps_per_epoch=25,
          epochs=epochs,
          verbose=1)

Hyperparameter-2

In [42]:
batch_size = 128
num_classes = len(y_cat.toarray()[0])
epochs = 20

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

model = Sequential()
model.add(Conv2D(48, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(48, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(48, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.33))
model.add(Flatten())
model.add(Dense(24, activation='relu'))
model.add(Dropout(0.33))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()
model.fit_generator(image_gen.flow(x_train, y_train.toarray(), batch_size=batch_size),
          steps_per_epoch=25,
          epochs=epochs,
          verbose=1)

Hyperparameter-3

In [44]:
batch_size = 256
num_classes = len(y_cat.toarray()[0])
epochs = 40

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

model = Sequential()
model.add(Conv2D(48, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(48, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(48, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.33))
model.add(Flatten())
model.add(Dense(24, activation='relu'))
model.add(Dropout(0.33))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()
model.fit_generator(image_gen.flow(x_train, y_train.toarray(), batch_size=batch_size),
          steps_per_epoch=25,
          epochs=epochs,
          verbose=1)

In [45]:
batch_size = 512
num_classes = len(y_cat.toarray()[0])
epochs = 60

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

model = Sequential()
model.add(Conv2D(48, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(48, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(48, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.33))
model.add(Flatten())
model.add(Dense(24, activation='relu'))
model.add(Dropout(0.33))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()
model.fit_generator(image_gen.flow(x_train, y_train.toarray(), batch_size=batch_size),
          steps_per_epoch=25,
          epochs=epochs,
          verbose=1)

###The difference between Deep learning on Spark and Deep learning on tensorflow

Tensorflow will occupy all of the cores locally so using spark is mainly distribute all the work to a distributed environment. There is actually no difference between the result if you apply the model and because of the multi-thread functions, there might lead to some unexpected result. Also, the APIs provided by Spark is not sufficient enough and compare to the tensorflow, it's not so easily understood and require a lot of pre-processing work which is quite confusing and the dataframe is quite different from the commonly used type. The databricks do not provide strong hardware such as GPU to support the training process and will automatically shut down. We can conclude that for simple deep learning, using Spark is not very comfortable, we will apply our model locally or run it on specified machines.